<a href="https://colab.research.google.com/github/Mastercheifs/docs-l10n/blob/master/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5_%EA%B0%95%ED%99%94%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import numpy as np
from collections import deque
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomUniform
import tensorflow as tf

class NN(tf.keras.Model):
  def __init__(self, action_size):
    super(NN, self).__init__()
    self.fc1 = Dense(24, activation='relu')
    self.fc2 = Dense(24, activation='relu')
    self.fc_out = Dense(action_size, kernel_initializer=RandomUniform(-1e-3, 1e-3))

  def call(self, x):
    x = self.fc1(x)
    x = self.fc2(x)
    q = self.fc_out(x)
    return q

class DQN:
  def __init__(self, state_size, aciton_size):
    self.state_size = state_size
    self.action_size = aciton_size

    self.discount_factor = 0.99
    self.learning_rate = 0.001
    self.epsilon = 1.0
    self.epsilon_decay = 0.999
    self.epsilon_min = 0.001
    self.batch_size = 64
    self.train_start = 1000

    self.memory = deque(maxlen=2000)

    self.model = NN(self.action_size)
    self.target_model = NN(self.action_size)
    self.optimizer = Adam(learning_rate=self.learning_rate)

    self.update_target_model()

  def update_target_model(self):
    self.target_model.set_weights(self.model.get_weights())

  def get_action(self, state):
    if np.random.rand() <= self.epsilon:
      return random.randrange(self.action_size)
    else:
      q = self.model(state) #리스트 형태로 반환됨
      return np.argmax(q[0])

  def append_sample(self, state, action, reward, next_state, done):
    self.memory.append((state, action, reward, next_state, done))

  def train_model(self):
    if self.epsilon > self.epsilon_min:
      self.epsilon *= self.epsilon_decay

    mini_batch = random.sample(self.memory, self.batch_size)
    states = np.array([sample[0][0] for sample in mini_batch])
    actions = np.array([sample[1] for sample in mini_batch])
    rewards = np.array([sample[2] for sample in mini_batch])
    next_states = np.array([sample[3][0] for sample in mini_batch])
    dones = np.array([sample[4] for sample in mini_batch])

    model_params = self.model.trainable_variables

    with tf.GradientTape() as tape:
      predicts = self.model(states)
      one_hot_action = tf.one_hot(actions , self.action_size)
      predicts = tf.reduce_sum(one_hot_action * predicts, axis=1)

      target_predicts = self.target_model(next_states)
      target_predicts = tf.stop_gradient(target_predicts)

      max_q = np.amax(target_predicts, axis=-1)
      targets = rewards + (1-dones) * self.discount_factor * max_q
      loss = tf.reduce_mean(tf.square(targets - predicts))

    grads = tape.gradient(loss, model_params)
    self.optimizer.apply_gradients(zip(grads, model_params))

In [ ]:
import numpy as np
import gym 
env = gym.make('Acrobot-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

agent = DQN(state_size, action_size)

scores, episodes = [], []
score_avg = 0

EPISODES = 300

for e in range(EPISODES):
  done = False
  score = 0

  state = env.reset()
  state = np.reshape(state, [1, state_size])

  while not done:
    action = agent.get_action(state)

    next_state, reward, done, info = env.step(action)
    next_state = np.reshape(next_state, [1, state_size])

    reward = -(next_state[0][0]) - (next_state[0][0]*next_state[0][1] - next_state[0][2]*next_state[0][3])
    score += reward

    agent.append_sample(state, action, reward, next_state, done)

    if len(agent.memory) >= agent.train_start:
      agent.train_model()

    state = next_state

    if done:
      agent.update_target_model()

      scores.append(score)
      episodes.append(e)

      if e % 10 == 0:  # 매 10 에피소드마다 평균 점수 계산
        score_avg = np.mean(scores[-min(10, len(scores)):])
        print(f"에피소드 : {e} | 점수 : {score} | 평균 점수 : {score_avg}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


에피소드 : 0 | 점수 : -471.9214314520359 | 평균 점수 : -471.9214314520359


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(episodes, scores)
plt.title('Score over Episodes')
plt.xlabel('literations')
plt.ylabel('Average Return')
plt.show()

In [ ]:
env_name = 'Acrobot-v1'
env = gym.make('Acrobot-v1')

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

In [ ]:
env.reset()
PIL.Image.fromarray(env.render(mode='rgb_array'))


In [ ]:
def policy(observation):
    return agent.get_action(observation)

create_policy_eval_video(policy, "trained-agent")


In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

In [ ]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

create_policy_eval_video(agent.policy, "trained-agent")

In [ ]:
create_policy_eval_video(random_policy, "random-agent")

In [ ]:
print(dir(agent))
